In [1]:
from helper_functions import *
import plotly.graph_objects as go
import pandas as pd
import wandb

In [2]:

from datetime import datetime
now = datetime.now()
datetime_str = now.strftime("%m/%d_%H:%M:%S")
datetime_str
from datetime import datetime
now = datetime.now()
datetime_str = now.strftime("%m/%d_%H:%M:%S")
datetime_str


'01/22_18:23:39'

In [6]:

# data_df = pd.read_pickle("Kris_updated_yolov8.pkl")
data_df = preprocess_data(pd.read_pickle("Kris_merged_yolov8.pkl"))
# data_df = preprocess_data(kris_df)
# Load current experiment number
try:
    with open('experiment_counter.txt', 'r') as f:
        experiment_num = int(f.read())
except:
    experiment_num = 1

# Increment and save the updated experiment number
with open('experiment_8_counter.txt', 'w') as f:
    f.write(str(experiment_num + 1))

In [5]:
kris_df

,timestamp_true,car_count,num_timestamp,month_no,month_name,day,day_of_week,day_of_week_name,is_weekend,hour_of_day,minutes,min_of_day,week_no,combined,centered_car_count
0,2023-06-06 20:36:00,0,28101396,6,June,6,1,Tuesday,0,20,36,1236,1,06062036,-12.493719
1,2023-06-06 21:06:00,9,28101426,6,June,6,1,Tuesday,0,21,6,1266,1,06062106,-3.493719
2,2023-06-06 21:37:00,0,28101457,6,June,6,1,Tuesday,0,21,37,1297,1,06062137,-12.493719
3,2023-06-06 22:07:00,7,28101487,6,June,6,1,Tuesday,0,22,7,1327,1,06062207,-5.493719
4,2023-06-06 22:37:00,2,28101517,6,June,6,1,Tuesday,0,22,37,1357,1,06062237,-10.493719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1826,2023-10-30 10:13:00,24,28311013,10,October,30,0,Monday,0,10,13,613,0,10301013,11.506281
1827,2023-10-30 10:18:00,28,28311018,10,October,30,0,Monday,0,10,18,618,0,10301018,15.506281
1828,2023-10-30 11:04:00,28,28311064,10,October,30,0,Monday,0,11,4,664,0,10301104,15.506281
1829,2023-10-30 11:18:00,39,28311078,10,October,30,0,Monday,0,11,18,678,0,10301118,26.506281


<h2 >EXP 8:  Expanding window Forecast origin cross validation:

<h3> define splits


In [7]:

train,valid ,test = split_data(data_df, train_ratio= 0.8,valid_ratio=0)
train.head()
len(train),len(valid),len(test)


(1464, 0, 367)


<h3> kernel definition


In [8]:

from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ExpSineSquared, ConstantKernel, Exponentiation as C

# RBF kernel with a length scale of 1.0
k1 = 1 * RBF(length_scale=0.5, length_scale_bounds=(1e-3, 1e3))

# White kernel to account for noise
k2 = WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))

# Periodic kernel
k3 = ExpSineSquared(length_scale=1.0,
                    length_scale_bounds=(1e-1, 50) ,
                    periodicity=1,
                    periodicity_bounds= (1e-1, 50))

k4 = k1 * k3
# Combined kernel
# kernel = k1+k2+k3
# kernel = k2 +k4
# kernel = k2*k4
# kernel = k4 + k3 +k1
kernel = (k4 + k2)+k3

gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer= 10)




<h3> Train,validate, test

In [22]:

# datetime_str = now.strftime("%m/%d_%H:%M:%S")


# # Initialize wandb run
# run = wandb.init(project="Expanding_Window_GPR", name=f"Expanding_Window_{experiment_num}_feature:hour_of_day")
# # Configuration parameters (this could be extended depending on what you'd like to track)
# config = wandb.config
# config.ini_train_win_len = 24
# config.valid_win_len = 24
# config.step_size = 1 # 24 hours or 1 day

# # Call the expanding window function
# y_pred, y_std, gpr = expanding_window_gpr_process(
#                                                 run,
#                                                 train,
#                                                 gpr,
#                                                 test,
#                                                 ini_train_win_len=config.ini_train_win_len, 
#                                                 valid_win_len=config.valid_win_len,
#                                                 features='hour_of_day',
#                                                 target='centered_car_count',
#                                                 agg_func='mean'
#                                                 )

# # Close the wandb run
# wandb.finish()


In [9]:

datetime_str = now.strftime("%m/%d_%H:%M:%S")

# Initialize wandb run
run = wandb.init(project="Expanding_Window_GPR",
                 name=f"Expanding_Window_{experiment_num}_feature:week")
# Configuration parameters (this could be extended depending on what you'd like to track)
config = wandb.config
config.ini_train_win_len = 24
config.valid_win_len = 24
config.step_size = 1 # 24 hours or 1 day

# Call the expanding window function
y_pred, y_std, gpr = expanding_window_gpr_process(
                                                run,
                                                train,
                                                gpr,
                                                test,
                                                ini_train_win_len=config.ini_train_win_len, 
                                                valid_win_len=config.valid_win_len,
                                                features='day_of_week',
                                                target='centered_car_count',
                                                agg_func='mean'
                                                )

# Close the wandb run
wandb.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: afn-sy. Use `wandb login --relogin` to force relogin


Processing expanding window:   1%|▎                                 | 1/120 [00:00<00:17,  6.75it/s]/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

Processing expanding window:   2%|▌                                 | 2/120 [00:00<00:53,  2.20it/s]/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.1. Decreasing the bound and calling fit again may find a better value.

Processing expanding window:   2%|▊                                 | 3/120 [00:00<00

Step: 1


Processing expanding window:   3%|█▏                                | 4/120 [00:01<00:39,  2.94it/s]

Step: 2


...

Step: 59


/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k1__k2__length_scale is close to the specified upper bound 50.0. Increasing the bound and calling fit again may find a better value.

Processing expanding window: 100%|████████████████████████████████| 120/120 [06:30<00:00,  3.26s/it]


Step: 60
Testing final GPR on test set


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


LML_test,▁
LML_train,██▇▅▃▅▃▂▃▃▂▃▃▃▃▄▃▄▃▃▂▄▄▂▁▁▁▁▁▁▁▁▁▃▄▄▄▄▄▃
LML_validate,██▇▅▃▅▃▂▃▃▂▃▃▃▃▃▃▂▃▃▂▂▄▂▁▁▁▁▁▁▁▁▁▁▄▄▄▄▄▃
MAE_test,▁
MAE_train,▁▁▁▁█▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▄▁▅▁▁▁▁▁▁▁▁▁▁▁
MAE_validate,▃▃▃▂▂▃▄▂▂▂▂▁▄▂▄▄▁▂▃▁▄█▇▇▄▃▄▄▃▄▄▆▃▅▄▅▂▅▃▂
MAPE_test,▁
MAPE_train,▁▁▁▁▆▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁█▁▇▁▁▁▁▁▁▁▁▁▁▁
MAPE_validate,▂▁▁▁▁▄▆▁▁▁▁▁▂▄▁▁▁▁▁▁▂▁▁▂▁▁▁▂▂▁▁▁▁▂▂▁▁▁▁█
MSE_test,▁
MSE_train,▁▁▁▁█▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▆▁▁▁▁▁▁▁▁▁▁▁


In [24]:

datetime_str = now.strftime("%m/%d_%H:%M:%S")


# Initialize wandb run
run = wandb.init(project="Expanding_Window_GPR", name=f"Expanding_Window_{experiment_num}_feature:month")
# Configuration parameters (this could be extended depending on what you'd like to track)
config = wandb.config
# config.ini_train_win_len = 24
config.ini_train_win_len = 24 * 7
config.valid_win_len = 24
# config.valid_win_len = 24 *7
config.step_size = 1 # 24 hours or 1 day

# Call the expanding window function
y_pred, y_std, gpr = expanding_window_gpr_process(
                                                run,
                                                train,
                                                gpr,
                                                test,
                                                ini_train_win_len=config.ini_train_win_len, 
                                                valid_win_len=config.valid_win_len,
                                                features='day',
                                                target='centered_car_count',
                                                agg_func='mean'
                                                )

# Close the wandb run
wandb.finish()


Processing expanding window:   2%|▋                                  | 1/50 [00:00<00:08,  5.49it/s]/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

Processing expanding window:   4%|█▍                                 | 2/50 [00:00<00:14,  3.38it/s]/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k1__k2__length_scale is close to the specified lower bound 0.1. Decreasing the bound and calling fit again may find a better value.

/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/

Step: 1


Processing expanding window:   8%|██▊                                | 4/50 [00:01<00:13,  3.51it/s]

Step: 2


/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k1__k2__length_scale is close to the specified lower bound 0.1. Decreasing the bound and calling fit again may find a better value.

Processing expanding window:  12%|████▏                              | 6/50 [00:01<00:14,  3.10it/s]/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k1__k1__k2__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.

/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k2__noise_level is close to the specifi

...

Step: 24


/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.1. Decreasing the bound and calling fit again may find a better value.

Processing expanding window: 100%|██████████████████████████████████| 50/50 [00:18<00:00,  2.75it/s]


Step: 25
Testing final GPR on test set


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


LML_test,▁
LML_train,██▇▇▇▇▆▆▅▄▄▄▄▄▃▃▂▂▁▁▁▁▁▁▁▁
LML_validate,██▇▇▇▇▆▆▅▄▄▄▄▄▃▃▂▂▁▁▁▁▁▁▁
MAE_test,▁
MAE_train,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▂▁▁▁▁▁▁
MAE_validate,▃▄▄▄▂▅▄▅▄█▂▂▁▂▅▅▁▄▁▂▁▂▁▂▄
MAPE_test,▁
MAPE_train,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▂▁▁▁▁▁▁
MAPE_validate,▂▂█▂▂▂▂▂▂▂▁▁▁▂▅▂▁▁▁▁▁▁▁▁█
MSE_test,▁
MSE_train,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁
